In [1]:
import numpy as np, pandas as pd, os
from tqdm import tqdm
df_toxic_comments = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
df_toxic_comments.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [2]:
df_toxic_comments.shape
df_toxic_comments['targets'] = (df_toxic_comments[['toxic', 'severe_toxic','obscene','threat','insult','identity_hate']].sum(axis=1)>0).astype(int)
df_toxic_comments.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,targets
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0


In [3]:
del df_toxic_comments['toxic'], df_toxic_comments['severe_toxic'], df_toxic_comments['obscene'], df_toxic_comments['threat'], df_toxic_comments['insult'], df_toxic_comments['identity_hate']
df_toxic_comments.head()

,id,comment_text,targets
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df_severe_toxic = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
df_severe_toxic.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


# Data preprocessing

In [5]:
_1 = pd.DataFrame()
_1['comments'] = df_toxic_comments['comment_text']
_1['targets'] = df_toxic_comments['targets']

_2 = pd.DataFrame()
_2['comments'] = df_severe_toxic['more_toxic']
_2['targets'] = 1

_3 = pd.DataFrame()
_3['comments'] = df_severe_toxic['less_toxic']
_3['targets'] = 1

_1 = _1.append(_2, ignore_index=True)
_1 = _1.append(_3, ignore_index=True)
raw_data = _1
raw_data.shape

(219787, 2)

In [6]:
raw_data.head()

,comments,targets
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
raw_data.drop_duplicates(subset='comments', keep='first', inplace=True)
raw_data.head()

,comments,targets
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
raw_data.shape

(166193, 2)

In [9]:
df = raw_data[(raw_data.targets == 1)]
len(df.comments.unique())
# gives us the actual counts of labels with 1

22847

In [10]:
from imblearn.under_sampling import RandomUnderSampler
X = np.array(raw_data['comments'].values)
X = X.reshape(-1,1)
Y = np.array(raw_data['targets'].values)
rus = RandomUnderSampler(random_state=0)
train, target = rus.fit_resample(X,Y)

train = train.flatten()
final_dataframe = pd.DataFrame()
final_dataframe['comments'] = train
final_dataframe['targets'] = target

final_dataframe.head()

,comments,targets
0,"""I do not understand this sentence: """"Several ...",0
1,"""\nThanks. I don't really mind the attacks. My...",0
2,""", 29 October 2007 (UTC)\n\nThis is a """"spinou...",0
3,2010 Formula One season,0
4,"""Welcome!\n\n \n\nHello, , to Wikipedia! I'm ,...",0


In [11]:
final_dataframe.targets.value_counts()

0    22847
1    22847
Name: targets, dtype: int64

In [12]:
import re
import string
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
lemma = WordNetLemmatizer()

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM, Dropout, BatchNormalization
from keras.layers.embeddings import Embedding

from tqdm import tqdm

In [13]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'',text)

def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

In [14]:
print('Applying preprocessing functions on final dataframe')
final_dataframe['comments'] = [remove_url(final_dataframe.comments[i]) for i in tqdm(range(final_dataframe.shape[0]))]
final_dataframe['comments'] = [remove_emoji(final_dataframe.comments[i]) for i in tqdm(range(final_dataframe.shape[0]))]
final_dataframe['comments'] = [remove_html(final_dataframe.comments[i]) for i in tqdm(range(final_dataframe.shape[0]))]
final_dataframe['comments'] = [remove_punct(final_dataframe.comments[i]) for i in tqdm(range(final_dataframe.shape[0]))]
final_dataframe.head()

Applying preprocessing functions on final dataframe


100%|██████████| 45694/45694 [00:01<00:00, 37767.00it/s]


,comments,targets
0,I do not understand this sentence Several auth...,0
1,\nThanks I dont really mind the attacks My nei...,0
2,29 October 2007 UTC\n\nThis is a spinout arti...,0
3,2010 Formula One season,0
4,Welcome\n\n \n\nHello to Wikipedia Im one of...,0


In [15]:
final_dataframe['comments'] = [word_tokenize(final_dataframe.comments[i]) for i in tqdm(range(final_dataframe.shape[0]))]
final_dataframe.head()

100%|██████████| 45694/45694 [00:30<00:00, 1517.42it/s]


,comments,targets
0,"[I, do, not, understand, this, sentence, Sever...",0
1,"[Thanks, I, dont, really, mind, the, attacks, ...",0
2,"[29, October, 2007, UTC, This, is, a, spinout,...",0
3,"[2010, Formula, One, season]",0
4,"[Welcome, Hello, to, Wikipedia, Im, one, of, t...",0


In [16]:
# Converting the text to lowercase
final_dataframe['comments'] = final_dataframe['comments'].apply(lambda x:[word.lower() for word in x])

# removing stop words
final_dataframe['comments'] = final_dataframe['comments'].apply(lambda x:[word for word in x if word not in set(stopwords.words('english'))])
final_dataframe.head()

,comments,targets
0,"[understand, sentence, several, authors, criti...",0
1,"[thanks, dont, really, mind, attacks, neighbor...",0
2,"[29, october, 2007, utc, spinout, article, the...",0
3,"[2010, formula, one, season]",0
4,"[welcome, hello, wikipedia, im, one, thousands...",0


In [17]:
final_dataframe['comments'] = [' '.join(map(str,l)) for l in final_dataframe['comments']]
final_dataframe.head()

,comments,targets
0,understand sentence several authors criticised...,0
1,thanks dont really mind attacks neighbor small...,0
2,29 october 2007 utc spinout article therefore ...,0
3,2010 formula one season,0
4,welcome hello wikipedia im one thousands edito...,0


In [18]:
final_dataframe.to_csv('/kaggle/working/final_data.csv', index=False)
print('done')

done
